# Title: Development of an Automatic Sentiment Analysis Tool for Urdu Text on Social Media Platforms

# Phase 1: Text Preprocessing

# Step 1: Import Necessary Libraries

In [28]:
!pip install lughaatNLP

In [31]:
import pandas as pd
from LughaatNLP import LughaatNLP
textProcessor = LughaatNLP()


# Step 2: Load the Dataset

In [32]:
# Load the dataset
data_frame = pd.read_csv('urdu_sarcastic_dataset.csv')

# Display the first few rows of the dataset
print(data_frame.head(20))


                                            urdu_text  is_sarcastic  \
0   🤣😂😂 ہو لینے دے میری شادی فسادن ٹھیک ہے کوجی نہ...           1.0   
1   چل مہمانوں میں کھانا سرو کر چڑیل چاچی نوں دسدی...           1.0   
2   کامران خان آپکی دن بھریہ زمہ داری لگائی گئی اپ...           0.0   
3                                        نہیں پائین 😎           0.0   
4    `` مراد علی شاہ کے بھیس میں ڈی جی آئی ایس آئی...           1.0   
5         قابل اعتبار ہی اکثر قاتل اعتبار ہوتے ہیں 💔🔥           1.0   
6       انساں کو تھکا دیتا ہے سوچوں کا سفر بھی ... 🍁🥀           0.0   
7                               حامد میر صاحب ویلڈن👏😊           0.0   
8   یار وچارہ ویلا ہوندا ہے اس آرے لگا ہویا ہے😂😂 ت...           1.0   
9            یہ سمجھتے ہیں سارا پاکستان بیوقوف ھے 😂😂😂           1.0   
10                      تسی لڑاںٔی کروانی ساڈی کی 😂😂😂           0.0   
11                       پائن دوبارہ فالو کرئیے..؟؟😆🙄           0.0   
12  کتنی مہنگائی ہے الو دوسو روپے درجن کدو 80روپے ...           1.0   
13   😍

In [5]:
pip install emoji


Note: you may need to restart the kernel to use updated packages.


# Step 3: Stopwords removal


In [38]:

sentiment_carrying_stopwords = ["نہیں", "برا", "واہ", "بہت"]


def remove_stopwords_with_lughaat(text, sentiment_stopwords):

    filtered_text = textProcessor.remove_stopwords(text)
 
    filtered_text = " ".join([word for word in filtered_text.split() if word not in sentiment_stopwords])
    return filtered_text


data_frame['clean'] = data_frame['urdu_text'].apply(
    lambda x: remove_stopwords_with_lughaat(x, sentiment_carrying_stopwords) if pd.notnull(x) else x
)


print("\nSample Data After Stopword Removal:")
print(data_frame[['urdu_text', 'clean']].head()
     
     )


Sample Data After Stopword Removal:
                                           urdu_text  \
0  🤣😂😂 ہو لینے دے میری شادی فسادن ٹھیک ہے کوجی نہ...   
1  چل مہمانوں میں کھانا سرو کر چڑیل چاچی نوں دسدی...   
2  کامران خان آپکی دن بھریہ زمہ داری لگائی گئی اپ...   
3                                       نہیں پائین 😎   
4   `` مراد علی شاہ کے بھیس میں ڈی جی آئی ایس آئی...   

                                               clean  
0                 🤣😂😂 لینے شادی فسادن ٹھیک کوجی 😐😐😐🤣  
1   چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی آں میں😂😂  
2  کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...  
3                                            پائین 😎  
4       `` مراد علی شاہ بھیس ڈی آئی آئی '' حامد میر😁  


# Step 4: Text Cleansing

In [42]:
import re
import string
import emoji

emoji_sentiment_dict = {
    "😊": "positive", "😢": "negative", "😂": "positive", "🤣": "positive",
    "😎": "positive", "😡": "negative", "❤️": "positive", "💔": "negative",
    "👍": "positive", "👎": "negative", "😭": "negative", "😜": "positive",
    "😁": "positive", "🔥": "positive", "😐": "negative"
}


def clean_text_with_lughaat(text, emoji_dict):
    if not isinstance(text, str):  
        return ''  
    
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove hashtags (optional)
    text = re.sub(r"#\w+", '', text)
    # Remove punctuations (except those in the sentiment dict)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Replace recognized emojis with their sentiment labels
    cleaned_text = []
    for char in text:
        if char in emoji_sentiment_dict:
            cleaned_text.append(emoji_sentiment_dict[char])  # Replace with sentiment label
        elif not emoji.is_emoji(char):  # Check if it's not an emoji
            cleaned_text.append(char)  # Keep original character

    return ''.join(cleaned_text)  
# Apply cleaning function
data_frame['cleaned_text1'] = data_frame['clean'].apply(
    lambda x: clean_text_with_lughaat(str(x), emoji_sentiment_dict) if pd.notnull(x) else x
)

# Print the results
print(data_frame['cleaned_text1'].head(50))

0     positivepositivepositive لینے شادی فسادن ٹھیک ...
1     چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی آں میں...
2     کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...
3                                        پائین positive
4        مراد علی شاہ بھیس ڈی آئی آئی  حامد میرpositive
5              قابل اعتبار قاتل اعتبار negativepositive
6                              انساں تھکا سوچوں سفر    
7                                حامد میر ویلڈنpositive
8     یار وچارہ ویلا ہوندا آرے ہویا ہےpositivepositi...
9     سمجھتے سارا پاکستان بیوقوف ھے positivepositive...
10      تسی لڑاںٔی کروانی ساڈی positivepositivepositive
11                        پائن دوبارہ فالو کرئیے   ؟ ؟ 
12    کتنی مہنگائی الو دوسو روپے درجن کدو 80روپے گز ...
13    عشق راس آۓ negativeزخم کھاٶ گے positiveمُسکراٶ گے
14                                        چونا positive
15    خاتم  النبیین  محمدﷺ Surat 73 سورة المزمل Ayt ...
16                بیچارے بیویاں ہیے تیسری ٹرائ positive
17                پتہ ہورہا بورڈ زنگ ہےnegativen

# Step 5: Applying the filtering function

In [44]:
def filter_short_posts(text):
    return len(text.split()) >= 3  # Keep posts with 3 or more words

# Apply the filtering function
data_frame['filtered_text'] = data_frame['cleaned_text1'].apply(lambda x: x if filter_short_posts(x) else None)

# Drop rows with None values in 'filtered_text'
data_frame.dropna(subset=['filtered_text'], inplace=True)

# Display the filtered results
print(data_frame[['cleaned_text1', 'filtered_text']].head(20))


                                        cleaned_text1  \
0   positivepositivepositive لینے شادی فسادن ٹھیک ...   
1   چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی آں میں...   
2   کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...   
4      مراد علی شاہ بھیس ڈی آئی آئی  حامد میرpositive   
5            قابل اعتبار قاتل اعتبار negativepositive   
6                            انساں تھکا سوچوں سفر       
7                              حامد میر ویلڈنpositive   
8   یار وچارہ ویلا ہوندا آرے ہویا ہےpositivepositi...   
9   سمجھتے سارا پاکستان بیوقوف ھے positivepositive...   
10    تسی لڑاںٔی کروانی ساڈی positivepositivepositive   
11                      پائن دوبارہ فالو کرئیے   ؟ ؟    
12  کتنی مہنگائی الو دوسو روپے درجن کدو 80روپے گز ...   
13  عشق راس آۓ negativeزخم کھاٶ گے positiveمُسکراٶ گے   
15  خاتم  النبیین  محمدﷺ Surat 73 سورة المزمل Ayt ...   
16              بیچارے بیویاں ہیے تیسری ٹرائ positive   
17              پتہ ہورہا بورڈ زنگ ہےnegativenegative   
18              اللہ رحمتوں سائ

# Replacing 'urdu_text' column with 'filtered_text' column

In [45]:
# Replace 'urdu_text' column with 'filtered_text' column
data_frame['urdu_text'] = data_frame['filtered_text']
print("Final Preprocessed Urdu Text:")
print(data_frame[['urdu_text']].head(30))

Final Preprocessed Urdu Text:
                                            urdu_text
0   positivepositivepositive لینے شادی فسادن ٹھیک ...
1   چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی آں میں...
2   کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...
4      مراد علی شاہ بھیس ڈی آئی آئی  حامد میرpositive
5            قابل اعتبار قاتل اعتبار negativepositive
6                            انساں تھکا سوچوں سفر    
7                              حامد میر ویلڈنpositive
8   یار وچارہ ویلا ہوندا آرے ہویا ہےpositivepositi...
9   سمجھتے سارا پاکستان بیوقوف ھے positivepositive...
10    تسی لڑاںٔی کروانی ساڈی positivepositivepositive
11                      پائن دوبارہ فالو کرئیے   ؟ ؟ 
12  کتنی مہنگائی الو دوسو روپے درجن کدو 80روپے گز ...
13  عشق راس آۓ negativeزخم کھاٶ گے positiveمُسکراٶ گے
15  خاتم  النبیین  محمدﷺ Surat 73 سورة المزمل Ayt ...
16              بیچارے بیویاں ہیے تیسری ٹرائ positive
17              پتہ ہورہا بورڈ زنگ ہےnegativenegative
18              اللہ رحمتوں سائے مکمل صحت یاب فرمائے

# Phase 2: Stemming and Lemmatization for Urdu Text using LughaatNLP

# Stemming

In [50]:

def apply_stemming(text):
    stemmed_text = textProcessor.urdu_stemmer(text)
    return stemmed_text

# Apply stemming to the 'cleaned_text' column
data_frame['stemmed_text'] = data_frame['urdu_text'].apply(lambda x: apply_stemming(str(x)) if pd.notnull(x) else x)

# Display the first few rows after stemming
print("\nSample Data After Stemming:")
print(data_frame[['urdu_text', 'stemmed_text']].head())



Sample Data After Stemming:
                                           urdu_text  \
0  positivepositivepositive لینے شادی فسادن ٹھیک ...   
1  چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی آں میں...   
2  کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...   
4     مراد علی شاہ بھیس ڈی آئی آئی  حامد میرpositive   
5           قابل اعتبار قاتل اعتبار negativepositive   

                                        stemmed_text  
0  positivepositivepositive لینہ شادی فسادن ٹھیک ...  
1  چل مہمانا کھانا سرو چڑیل چاچی نا دسدی آں میںpo...  
2  کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...  
4      مراد علی شاہ بھیس ڈی آئی آئی حامد میرpositive  
5             قابل اعتبر قاتل اعتبر negativepositive  


In [51]:
data_frame['urdu_text'] = data_frame['stemmed_text']
print("Final Preprocessed Urdu Text:")
print(data_frame[['urdu_text']].head(30))

Final Preprocessed Urdu Text:
                                            urdu_text
0   positivepositivepositive لینہ شادی فسادن ٹھیک ...
1   چل مہمانا کھانا سرو چڑیل چاچی نا دسدی آں میںpo...
2   کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...
4       مراد علی شاہ بھیس ڈی آئی آئی حامد میرpositive
5              قابل اعتبر قاتل اعتبر negativepositive
6                                 انساں تھکا سوچا سفر
7                              حامد میر ویلڈنpositive
8   یار وچارہ ویلا ہوندا آرہ ہویا ہےpositivepositi...
9   سمجھتہ سارا پاکستن بیوقوف ھہ positivepositivep...
10    تسی لڑاںٔی کروانی ساڈی positivepositivepositive
11                          پائن دوبرہ فالو کرئیہ ؟ ؟
12  کتنی مہنگائی الو دوسو روپہ درجن کدو 80روپہ گز ...
13  عشق راس آۓ negativeزخم کھاٶ گہ positiveمُسکراٶ گہ
15  خاتم النبیین محمدﷺ Surat 73 سوره المزمل Ayt 20...
16                بیچارہ بیوی ہیہ تیسری ٹرائ positive
17              پتہ ہورہا بورڈ زنگ ہےnegativenegative
18                 اللہ رحمت سائہ مکمل صحت یاب فرمائ

# Lemmatization

In [54]:
def apply_lemmatization(text):
    lemmatized_text = textProcessor.lemmatize_sentence(text)
    return lemmatized_text

# Apply lemmatization to the 'stemmed_text' column
data_frame['lemmatized_text'] = data_frame['stemmed_text'].apply(lambda x: apply_lemmatization(str(x)) if pd.notnull(x) else x)

# Display the first few rows after lemmatization
print("\nSample Data After Lemmatization:")
print(data_frame[['stemmed_text', 'lemmatized_text']].head())

data_frame['urdu_text'] = data_frame['lemmatized_text']
print("Final Preprocessed Urdu Text:")
print(data_frame[['urdu_text']].head(30))


Sample Data After Lemmatization:
                                        stemmed_text  \
0  positivepositivepositive لینہ شادی فسادن ٹھیک ...   
1  چل مہمانا کھانا سرو چڑیل چاچی نا دسدی آں میںpo...   
2  کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...   
4      مراد علی شاہ بھیس ڈی آئی آئی حامد میرpositive   
5             قابل اعتبر قاتل اعتبر negativepositive   

                                     lemmatized_text  
0  positivepositivepositive لینہ شادی فسادن ٹھیک ...  
1  چلنا مہمانا کھا سرا چڑیل چاچی نا دسدی آں میںpo...  
2  کامران خان دن بھریہ زمہ داری لگنا جانا اپوزیشن...  
4      مراد علی شاہ بھیس ڈی آئی آئی حامد میرpositive  
5             قابل اعتبر قاتل اعتبر negativepositive  
Final Preprocessed Urdu Text:
                                            urdu_text
0   positivepositivepositive لینہ شادی فسادن ٹھیک ...
1   چلنا مہمانا کھا سرا چڑیل چاچی نا دسدی آں میںpo...
2   کامران خان دن بھریہ زمہ داری لگنا جانا اپوزیشن...
4       مراد علی شاہ بھیس ڈی آئی آئی حامد میرpositive

# Phase 3: Feature Extraction from Urdu Text using LughaatNLP and Scikit-learn

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec

# Step 1: Tokenization

In [57]:
# Function to tokenize the text using LughaatNLP's urdu_tokenize function
def tokenize_text(text):
    tokens = textProcessor.urdu_tokenize(text)
    return tokens

# Apply tokenization to the 'lemmatized_text' column
data_frame['tokenized_text'] = data_frame['lemmatized_text'].apply(lambda x: tokenize_text(str(x)) if pd.notnull(x) else x)

# Display the first few rows after tokenization
print("\nSample Data After Tokenization:")
print(data_frame[['urdu_text', 'tokenized_text']].head())


Sample Data After Tokenization:
                                           urdu_text  \
0  positivepositivepositive لینہ شادی فسادن ٹھیک ...   
1  چلنا مہمانا کھا سرا چڑیل چاچی نا دسدی آں میںpo...   
2  کامران خان دن بھریہ زمہ داری لگنا جانا اپوزیشن...   
4      مراد علی شاہ بھیس ڈی آئی آئی حامد میرpositive   
5             قابل اعتبر قاتل اعتبر negativepositive   

                                      tokenized_text  
0                    [لینہ, شادی, فسادن, ٹھیک, کوجی]  
1  [چلنا, مہمانا, کھا, سرا, چڑیل, چاچی, نا, دسدی,...  
2  [کامران, خان, دن, بھریہ, زمہ, داری, لگنا, جانا...  
4    [مراد, علی, شاہ, بھیس, ڈی, آئی, آئی, حامد, میر]  
5                         [قابل, اعتبر, قاتل, اعتبر]  


# Step 2: TF-IDF (Term Frequency-Inverse Document Frequency) Feature Extraction

In [62]:
# Define the TF-IDF Vectorizer for Urdu text
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(), max_features=100)

# Fit and transform the lemmatized text to extract TF-IDF features
tfidf_matrix = tfidf_vectorizer.fit_transform(data_frame['urdu_text'])

# Convert the TF-IDF matrix to a DataFrame for better readability
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Display the top 10 words with the highest average TF-IDF scores
top_tfidf_words = tfidf_df.mean().sort_values(ascending=False).head(10)
print("\nTop 10 Words with Highest TF-IDF Scores:")
print(top_tfidf_words)


Top 10 Words with Highest TF-IDF Scores:
۔                           0.133195
positive                    0.080712
positivepositive            0.035554
positivepositivepositive    0.028631
؟                           0.028484
اللہ                        0.027203
کوئی                        0.027128
خان                         0.025684
به                          0.023354
ھہ                          0.022596
dtype: float64


/Applications/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Step 3: Word2Vec Model Training and Similarity Analysis

In [69]:
# Prepare the tokenized text for Word2Vec training
tokenized_texts =data_frame['tokenized_text'].tolist()

# Train the Word2Vec model using the tokenized text
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=8, min_count=2, workers=4)

# List the top 5 words most similar to the word "اچھا" (good)
try:
    similar_words = word2vec_model.wv.most_similar("اچھا", topn=5)
    print("\nTop 5 Words Most Similar to 'اچھا':")
    for word, similarity in similar_words:
        print(f"{word}: {similarity:.4f}")
except KeyError:
    print("\nThe word 'اچھا' is not in the vocabulary.")


Top 5 Words Most Similar to 'اچھا':
بچہ: 0.9874
حال: 0.9860
آئہ: 0.9840
سوچ: 0.9822
بت: 0.9810


# Phase 4: N-grams Analysis

In [71]:
import nltk
from nltk.util import ngrams
from collections import Counter


tokenized_texts = data_frame['tokenized_text'].tolist()  

# Flatten the tokenized texts for unigram analysis
all_words = [word for tokens in tokenized_texts for word in tokens]

# Unigrams
unigram_freq = Counter(all_words)

# Bigrams
bigram_list = [list(ngrams(tokens, 2)) for tokens in tokenized_texts]
bigrams = [bigram for sublist in bigram_list for bigram in sublist]  # Flatten
bigram_freq = Counter(bigrams)

# Trigrams
trigram_list = [list(ngrams(tokens, 3)) for tokens in tokenized_texts]
trigrams = [trigram for sublist in trigram_list for trigram in sublist]  # Flatten
trigram_freq = Counter(trigrams)



# Get the top 10 most common unigrams, bigrams, and trigrams

In [72]:


top_unigrams = unigram_freq.most_common(10)
top_bigrams = bigram_freq.most_common(10)
top_trigrams = trigram_freq.most_common(10)

# Display the results
print("\nTop 10 Unigrams:")
for word, freq in top_unigrams:
    print(f"{word}: {freq}")

print("\nTop 10 Bigrams:")
for bigram, freq in top_bigrams:
    print(f"{bigram}: {freq}")

print("\nTop 10 Trigrams:")
for trigram, freq in top_trigrams:
    print(f"{trigram}: {freq}")


Top 10 Unigrams:
۔: 11547
؟: 1580
️: 1560
کوئی: 1174
خان: 1172
ھہ: 1067
اللہ: 989
به: 976
پاکستن: 859
سندھ: 815

Top 10 Bigrams:
('۔', '۔'): 4685
('عمران', 'خان'): 508
('؟', '؟'): 496
('نواز', 'شریف'): 448
('\u200d', '️'): 332
('سندھ', 'پولیس'): 307
('️', '️'): 260
('آرمی', 'چیف'): 223
('کیپٹن', 'صفدر'): 179
('اردو', 'زبن'): 172

Top 10 Trigrams:
('۔', '۔', '۔'): 2302
('؟', '؟', '؟'): 234
('\u2066', '️', '\u2069'): 131
('️', '️', '️'): 121
('\u200d', '️', '\u200d'): 117
('️', '\u200d', '️'): 116
('پینا', 'ٹی', 'آئی'): 114
('پینا', 'ڈی', 'ایم'): 88
('صلی', 'اللہ', 'علیہ'): 88
('جزاک', 'اللہ', 'خیر'): 74


# Phase 5: Sentiment Classification Model

# tf-idf

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'labels' column has the sentiment labels (1 for positive, 0 for negative/neutral)
X = tfidf_matrix
y = data_frame['is_sarcastic']  # replace with your actual label column

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Logistic Regression model
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Predict on the test set
y_pred = log_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.7126
Precision: 0.7428
Recall: 0.7095
F1 Score: 0.7258


Evaluation Summary
Accuracy: 71.26%
The model correctly classified approximately 71 out of 100 instances in the validation set.
Precision: 74.28%
Of all the positive predictions made by the model, 74.28% were accurate, indicating a reduced number of false positives compared to previous models.
Recall: 70.95%
The model identified 70.95% of the actual positive sentiments, showing it maintains good coverage of relevant instances.
F1 Score: 72.58%
This score reflects a balance between precision and recall, indicating an overall improvement in performance compared to earlier results.

# 1. Evaluation
#Model Performance
The sentiment analysis model achieved an accuracy of 71.26%, with precision at 74.28%, recall at 70.95%, and an F1 score of 72.58%. These metrics indicate that the model is fairly balanced, with a good ability to identify positive sentiments but showing some difficulty with negative sentiments. For instance, it may misinterpret nuanced phrases like “یہ بہت برا ہے” (This is very bad) if it fails to grasp the context or the negation, leading to inaccuracies in sentiment classification.
# Challenges with Stemming and Lemmatization
Stemming and lemmatization are particularly challenging in Urdu due to the language’s complex morphology. Words can change forms based on gender, plurality, or tense. For example, the word "اچھا" (good) can appear as "اچھی" (good, feminine) and "اچھے" (good, plural). This variation can confuse the model, especially in contexts like “یہ اچھا نہیں ہے” (This is not good), where understanding negation is crucial for accurate sentiment analysis. If the model does not recognize these morphological changes, it may misclassify the overall sentiment.
# Areas for Improvement
To enhance performance, we should focus on refining preprocessing techniques, such as applying more advanced stemming algorithms that accommodate the richness of Urdu. Implementing methods like analysis and utilizing pre-trained models that consider context could significantly improve sentiment classification accuracy, especially for complex sentence structures.


# 2. Challenges in Urdu Sentiment Analysis
# Complex Morphology
Urdu’s intricate word forms pose significant challenges for stemming and lemmatization. For instance, the root “چلنا” (to walk) can manifest as “چلتا” (masculine), “چلتی” (feminine), and “چلیں گے” (will walk, plural). This complexity can lead to context loss during analysis, misclassifying sentiments in sentences such as “وہ ہمیشہ چلتی رہتی ہے” (She keeps walking always), where the sentiment may hinge on understanding the specific word forms used.
# Colloquial Language
The informal and colloquial nature of language used on social media introduces slang and abbreviations that complicate sentiment analysis. Phrases like “بہت زبردست” (very awesome) or “کمال ہے” (it's amazing) might not align with formal language models. Additionally, expressions such as “نہیں یار” (no man) can convey sarcasm but may be misinterpreted by the model as literal negation, leading to incorrect sentiment classification.
# Noisy Data
Social media platforms are filled with noisy data, including emojis, misspellings, and unconventional grammar. For example, a post containing “یہ تو بہت 😂 ہے” (This is so funny) may confuse the model due to the emotional context of the emoji. Typos, such as “برا” (bad) being written as “براہ” or “بڑا” (big), can further hinder the model's ability to extract clear sentiment signals.
# Optimizing the NLP Pipeline
To enhance Urdu sentiment analysis, we should implement better data normalization techniques, including spell-checking algorithms and expanding the training dataset to encompass a wider variety of colloquial expressions. Utilizing advanced language models, like transformers specifically designed for Urdu, can also capture contextual meanings more effectively. By addressing these challenges and adopting these strategies, we can significantly improve the accuracy and robustness of sentiment analysis in this rich language.
